In [1]:
import numpy as np
from tqdm.auto import tqdm
from load import load_data
from convert import convert
from pmc2 import *

In [2]:
data = load_data(13)
d = convert(data)

print(d['C'][:,3])
print(d['E'][:,3])
print(d['o'][:,3])

[ 9.9  8.3 14.8 13.4 21.  19.6 30.5 29.3 46.1 45.3 58.2 57.  69.8 68.5
 80.7 79.4 91.3 90.3]
[ 23.6  27.1  27.3  35.3  35.2  43.4  42.7  49.8  68.6  96.4 107.9]
[28.3 28.3 28.3 56.  56. ]


In [3]:
## INITIALIZATION
lin_C = d['pC'].size
lin_E = d['pE'].size
lin_o = d['po'].size

data_C = get_C(data)[np.argsort(get_C(data)[:, 3])] # sorted with p (mean stress)
data_E = get_E(data)[np.argsort(get_E(data)[:, 3])] # sorted with p (mean stress)
data_o = get_o(data)[np.argsort(get_o(data)[:, 5])] # sorted with teta (Lodge angle)

In [4]:
## ITERATIONS
if lin_C == 0 :
    mean_error, P1_E, P2_E, P1_o, P2_o, P1_C, P2_C = brute_force_two_sets(data_E,lin_E,data_o,lin_o,data_C)
if lin_E == 0 :
    mean_error, P1_C, P2_C, P1_o, P2_o, P1_E, P2_E = brute_force_two_sets(data_C,lin_C,data_o,lin_o,data_E)
if lin_o == 0 :
    mean_error, P1_C, P2_C, P1_E, P2_E, P1_o, P2_o = brute_force_two_sets(data_C,lin_C,data_E,lin_E,data_o)
if lin_C != 0 and lin_E != 0 and lin_o != 0 :
    mean_error, P1_C, P2_C, P1_E, P2_E, P1_o, P2_o = brute_force_three_sets(data_C,lin_C,data_E,lin_E,data_o,lin_o)


7.0 3.0 2.0


In [5]:
## RESULTS
print(np.nanmin(mean_error))
P1_init, P2_init = create_P1_and_P2_iteration(P1_C,P2_C,P1_E,P2_E,P1_o,P2_o)

if P1_init.Vo < P2_init.Vo :
    P1 = P2_init
    P2 = P1_init
elif P1_init.Vo > P2_init.Vo :
    P1 = P1_init
    P2 = P2_init
    
print(P1.sol)
print(P2.sol)

0.07566318839478585
[45.68424548 32.20162158 -1.06081639 79.47893648 15.20496742 12.5475467 ]
[13.72724253  9.5092449  -1.08953858  9.05169239 37.24904817 39.55115315]


In [6]:
## ERROR COMPUTATION

## Error based on Asig1+Bsig2+Csig3=1
P1_coeff = [P1.A,P1.B,P1.C]
P2_coeff = [P2.A,P2.B,P2.C]
sP1 = np.dot(np.transpose(P1.sig123),P1_coeff) #yi P1
sP2 = np.dot(np.transpose(P2.sig123),P2_coeff) #yi P2
sP = np.concatenate((sP1,sP2),axis=None) #yi
nb_err = sP.size
#print(sP)

# MAPE = (1/n)*sum((ymeas-ypred)/(ymeas))*100
#ymeas = sP
#ypred = 1
#Sm = 0
#for i in range(0,nb_err-1,1) :
#    Sm = Sm + abs((ymeas[i]-ypred)/ymeas[i])
#MAPE = (1/nb_err)*Sm*100
#print(MAPE)

## Error based on q computation
q_fitP1 = ((P1.bc)/(cos(P1.t)-P1.k*sin(P1.t)))*((P1.p/P1.Vo)+1)
q_fitP2 = ((P2.bc)/(cos(P2.t)-P2.k*sin(P2.t)))*((P2.p/P2.Vo)+1)
#print(q_fitP1)
#print(P1.q)
#print(q_fitP2)

f_P1 = q_fitP1
y_P1 = P1.q
f_P2 = q_fitP2
y_P2 = P2.q 
f = np.concatenate((f_P1,f_P2),axis=None)
err_P1 = y_P1-f_P1
err_P2 = y_P2-f_P2
err = np.concatenate((err_P1,err_P2),axis=None)
#print(err)

# R^2 = 1 - SSres/SStot
#SSres = Sum(yi-fi)^2
nb_err = err.size
#print(nb_err)
SSres = 0
for i in range(0,nb_err-1,1) :
    SSres = SSres + (err[i])**2
#print(SSres)
# SStot = Sum(yi-ymean)^2
ymean = np.mean(abs(err))
#print(ymean)
SStot = 0
for i in range(0,nb_err-1,1) :
    SStot = SStot + (f[i]-ymean)**2
#print(SStot)
# R^2 = 1 - SSres/SStot
Rsqu = 1 - SSres/SStot
print(Rsqu)


0.9971033628521127
